In [24]:
import requests 
import pandas as pd

# Raivenomace20 information
""" api_url_summoner = "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-puuid/oeUTUlTvQIJUxqO955b0viyfcv_-2zvQgTSdhgFJg1nTNJpPSMgtu65dKhC780TONNCU91gxAdNjdQ"
resp1 = requests.get(api_url_summoner)
player_info = resp1.json() """
puuid = "oeUTUlTvQIJUxqO955b0viyfcv_-2zvQgTSdhgFJg1nTNJpPSMgtu65dKhC780TONNCU91gxAdNjdQ"
mass_region = "americas"
# WILL NOT WORK AFTER TODAY. API CHANGES EVERYDAY UNLESS RIOT DEVELOPER WHICH I HAVE NOT BEEN APPROVED OF. SAMPLE STILL REMAINS
api_key = "RGAPI-41d30ce2-20ec-4039-a9a6-2db74100fdc7"
match_id = "NA1_5027023293"

# The function to get a list of all the match IDs (2nd example above) given a players puuid and mass region
def get_match_ids(puuid, mass_region, api_key):
    api_url_match = (
        "https://" +
        mass_region +
        ".api.riotgames.com/tft/match/v1/matches/by-puuid/" +
        puuid + 
        "/ids?start=0&count=20" + 
        "&api_key=" + 
        api_key
    )
    
    print(api_url_match)
    
    resp = requests.get(api_url_match)
    match_ids = resp.json()
    return match_ids

# From a given match ID and mass region, get the data about the game
def get_match_data(match_id, mass_region, api_key):
    api_url_matchdata = (
        "https://" + 
        mass_region + 
        ".api.riotgames.com/tft/match/v1/matches/" +
        match_id + 
        "?api_key=" + 
        api_key
    )
    
    # we need to add this "while" statement so that we continuously loop until it's successful
    while True:
        resp = requests.get(api_url_matchdata)
        
        # whenever we see a 429, we sleep for 10 seconds and then restart from the top of the "while" loop
        if resp.status_code == 429:
            print("Rate Limit hit, sleeping for 10 seconds")
            time.sleep(10)
            # continue means start the loop again
            continue
            
        # if resp.status_code isn't 429, then we carry on to the end of  the function and return the data
        match_data = resp.json()
        return match_data


def gather_all_data(puuid, match_ids, mass_region, api_key):
    # We initialise an empty dictionary to store data for each game
    data = {
        'placement': [],
        'level': [],
        'traits': [],
        'units': [],
        'items': []
    }
    
    for match_id in match_ids:
        # run the two functions to get the player data from the match ID
        match_data = get_match_data(match_id, mass_region, api_key)
        # assign the variables we're interested in
        index = None
        for pos, id in enumerate(match_data['metadata']['participants']):
            if puuid == id:
                index = pos
                break
        # print(puuid, match_data['metadata']['participants'][index])
        placement = match_data['info']['participants'][index]['placement']
        level = match_data['info']['participants'][index]['level']
        traits = [trait['name'] for trait in match_data['info']['participants'][index]['traits']]
        units = [unit['character_id'] for unit in match_data['info']['participants'][index]['units']]
        items = [item['itemNames'] for item in match_data['info']['participants'][index]['units']]
        # add them to our dataset
        data['placement'].append(placement)
        data['level'].append(level)
        data['traits'].append(traits)
        data['units'].append(units)
        data['items'].append(items)
    
    df = pd.DataFrame(data)
 
    return df

match_ids = get_match_ids(puuid, mass_region, api_key)
df = gather_all_data(puuid, match_ids, mass_region, api_key)
print(df.head())

https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/oeUTUlTvQIJUxqO955b0viyfcv_-2zvQgTSdhgFJg1nTNJpPSMgtu65dKhC780TONNCU91gxAdNjdQ/ids?start=0&count=20&api_key=RGAPI-41d30ce2-20ec-4039-a9a6-2db74100fdc7
   placement  level                                             traits  \
0          4      8  [TFT12_BatQueen, TFT12_Chrono, TFT12_Faerie, T...   
1          4      9  [TFT12_Arcana, TFT12_Ascendant, TFT12_Bastion,...   
2          1     10  [TFT12_Chrono, TFT12_Multistriker, TFT12_Shape...   
3          7      9  [TFT12_Arcana, TFT12_Bastion, TFT12_BatQueen, ...   
4          6      8  [TFT12_Arcana, TFT12_Bastion, TFT12_Eldritch, ...   

                                               units  \
0  [TFT12_Jax, TFT12_Zilean, TFT12_Vex, TFT12_Kar...   
1  [TFT12_Blitzcrank, TFT12_Ziggs, TFT12_Nunu, TF...   
2                                    [TFT12_Camille]   
3  [TFT12_Zilean, TFT12_Cassiopeia, TFT12_Akali, ...   
4  [TFT12_Nilah, TFT12_Akali, TFT12_Katarina, TFT...   

  

In [34]:
# TRAITS AND UNITS
# Initialize count dictionaries
trait_count = {}
unit_count = {}

# Loop through each row in the DataFrame where placement is above 4
for index, row in df[df['placement'] > 4].iterrows():
    # Count traits
    for trait in row['traits']:
        if trait in trait_count:
            trait_count[trait] += 1
        else:
            trait_count[trait] = 1

    # Count units
    for unit in row['units']:
        if unit in unit_count:
            unit_count[unit] += 1
        else:
            unit_count[unit] = 1

# Sort by descending and only if it appers more than 2 times
sorted_trait_count = sorted(
    ((trait, count) for trait, count in trait_count.items() if count > 2),
    key=lambda x: x[1],
    reverse=True)
sorted_unit_count = sorted(
    ((unit, count) for unit, count in unit_count.items() if count > 2),
    key=lambda x: x[1],
    reverse=True)

# Display the counts
print("Most common traits when placement is above 4:")
for trait, count in sorted_trait_count:
    print(f"{trait}: {count}")

print("\nMost common units when placement is above 4:")
for unit, count in sorted_unit_count:
    print(f"{unit}: {count}")

Most common traits when placement is above 4:
TFT12_Faerie: 9
TFT12_Pyro: 9
TFT12_Witchcraft: 9
TFT12_Multistriker: 8
TFT12_Bastion: 7
TFT12_Preserver: 7
TFT12_Shapeshifter: 7
TFT12_Warrior: 7
TFT12_Arcana: 6
TFT12_Chrono: 6
TFT12_Frost: 5
TFT12_BatQueen: 4
TFT12_Eldritch: 4
TFT12_Scholar: 4
TFT12_Sugarcraft: 3
TFT12_Portal: 3

Most common units when placement is above 4:
TFT12_Hecarim: 6
TFT12_Fiora: 6
TFT12_Rakan: 6
TFT12_Kalista: 6
TFT12_Zilean: 5
TFT12_Akali: 5
TFT12_Nasus: 5
TFT12_Morgana: 4
TFT12_Shen: 3
TFT12_Neeko: 3
TFT12_Gwen: 3
TFT12_Milio: 3


In [58]:
# ITEMS
from collections import defaultdict

# Initialize a dictionary to map units to items and count occurrences
unit_item_count = defaultdict(lambda: defaultdict(int))

# Populate the dictionary with units and their corresponding items
for index, row in df.iterrows():
    for unit, items in zip(row['units'], row['items']):
        for item in items:
            unit_item_count[unit][item] += 1

# Sort by descending
sorted_item_count = {unit: item_counts for unit, item_counts in unit_item_count.items() if unit in dict(sorted_unit_count)}
            
# Print the item counts for each unit
for unit, items in sorted_item_count.items():
    sorted_items = sorted(items.items(), key=lambda x: x[1], reverse=True)
    print(f"\nMost common items for unit '{unit}':")
    for item, count in sorted_items:
        print(f"{item}: {count}")



Most common items for unit 'TFT12_Rakan':
TFT_Item_WarmogsArmor: 3
TFT12_Item_Faerie_QueensCrown: 2
TFT_Item_Crownguard: 2
TFT_Item_IonicSpark: 1
TFT12_Item_WitchcraftEmblemItem: 1
TFT_Item_RedBuff: 1
TFT_Item_FrozenHeart: 1
TFT12_Item_Faerie_Armor: 1
TFT_Item_GargoyleStoneplate: 1
TFT_Item_ZekesHerald: 1
TFT_Item_SpectralGauntlet: 1
TFT_Item_BrambleVest: 1

Most common items for unit 'TFT12_Kalista':
TFT_Item_GuinsoosRageblade: 8
TFT12_Item_Faerie_QueensCrown: 6
TFT_Item_InfinityEdge: 4
TFT_Item_LastWhisper: 3
TFT_Item_Artifact_RapidFirecannon: 2
TFT9_Item_OrnnHorizonFocus: 1
TFT12_Item_WitchcraftEmblemItem: 1
TFT_Item_UnstableConcoction: 1
TFT_Item_RapidFireCannon: 1
TFT_Item_Deathblade: 1

Most common items for unit 'TFT12_Morgana':
TFT_Item_WarmogsArmor: 3
TFT_Item_GargoyleStoneplate: 2
TFT_Item_DragonsClaw: 2
TFT_Item_NightHarvester: 1
TFT_Item_ThiefsGloves: 1
TFT_Item_AdaptiveHelm: 1
TFT_Item_PowerGauntlet: 1
TFT4_Item_OrnnEternalWinter: 1
TFT_Item_AegisOfTheLegion: 1

Most comm